# Classes et héritage

## Heritage simple

- Créer une classe ModelUn  retournant une constante fixée lors de l'initialisation

In [1]:
import numpy as np

class ModelUn():
    def __init__(self, init_val:int = 42) -> None:
        # TODO
        print("Model 1 initialisation")
        
    def predict(self, X:np.ndarray) -> int:
        print("Model 1  prediction")
        return # TODO

In [ ]:
clf = ModelUn(43)
print("prediction : ",clf.predict(1))

remarque : la notation ci dessous est équivalente

In [ ]:
print("prediction : ",ModelUn.predict(clf,1))

- On souhaite étendre le comportement de la classe ModelUn pour un nouveau projet, sans modifier le code de l'existant. créer une seconde classe héritant de la premiere, stockant un entier supplémentaire lors de l'initilisation, et retournant la liste des deux entiers. 

Cette méthode appelle les méthodes du père, et complete leur comportement

In [ ]:
class ModelDeux(ModelUn):
    def __init__(self, init_val:int = 42, ini_val2:int=43) -> None:
        print("Model 2 initialisation")
        # TODO
        
    def predict(self, X:np.ndarray) -> int:
        print("Model 2 prediction")
        # TODO

In [ ]:
clf = ModelDeux(1,2)
print("--------")
clf.predict(1)

Remarque :
- super n'appelle pas le parent ! (pas toujours)
- super().predict(X) et ModelUn.predict(self, X) ont le meme effet dans l'exemple precedent
- les deux ne sont pas completement equivalent : la premiere laisse python trouver quelle est la methode, tandis que la deuxieme l'impose

## Heritage multiple

- On souhaite rajouter un entrainement sans modifier la classe existante. Le modèle stocke la valeur lue lors de l'entrainement, et la rajoute à la liste de la méthode prédict. On dispose déjà d'une classe faisant une partie du comportement décrite ci dessous :

In [ ]:
class ModelTrois(ModelUn):       
    def fit(self, X:int) -> None:
        print("model 3 fit")
        # TODO
            
    def predict(self, X:np.ndarray) -> int:
        print("model 3 prediction")
        first_value = # TODO
        return [first_value, self.learnt_value]

In [ ]:
clf = ModelTrois(3)
print("--------")
clf.fit(2)
print("--------")
clf.predict(1)

- HOUSTON, WE HAVE A PROBLEM

In [ ]:
class ModelQuatre(ModelDeux,ModelTrois):
    None

In [ ]:
clf = ModelQuatre(33,34)
print("--------")
clf.fit(2)
print("--------")
clf.predict(3)

La méthode predict() de model2 appelle en fait predict() de model3
Comment cela ce fait ?
- super sélectionne la classe appeler en fonction d'un ordre prédefini, le MRO (multiple resolution order)
- pour une classe, le MRO est déterminé par le type de l'instance, ici toujours celui de ModelQuatre

- en python, on peut afficher le mro de chaque classe :

In [ ]:
ModelQuatre.__mro__

La méthode super en vrai:
- super() est en fait super(ModelQuatre, self)
- le premier paramètre indique où on est dans le mro, le deuxieme l'objet auquel prendre le mro
- si le deuxieme parametre est une instance, on peut appeler predict(X).
- si le deuxieme parametre est une classe, il faut appeler predict(self,X)

Exemples :
- super(MySecondModel, self).predict(X) va choisir la méthode de MyThirdModel
- super(MySecondModel, MySecondModel).predict(self, X) va appeler A, car se place dans MySecondModel mais appelle le mro de MySecondModel

- ce n'est pas le comportement que l'on veut. Comment faire pour appeler d'un côté le modele 2 qui appelle 1, et de l'autre le modele 3 qui appelle le 1?

Solution : Changer le code de model2 pour ne pas laisser à super() le choix (ne pas le laisser appeler le mro de self) pour resoudre le nom :

In [ ]:
class ModelDeux(ModelUn):
    def __init__(self, init_val:int = 42, ini_val2:int=43) -> None:
        self.prediction_value2 = ini_val2
        print("Model 2 initialisation")
        super().__init__(init_val)
        
    def predict(self, X:np.ndarray) -> int:
        print("Model 2 prediction")
        first_value = # TODO
        # ligne equivalente : first_value = ModelUn.predict(self, X)
        return [first_value, self.prediction_value2]

In [ ]:
class ModelQuatre(ModelDeux, ModelTrois):
    def predict(self, X):
        res_model3 = # TODO # appelle 3 car le suivant selon le mro de self lorsque l'on est dans ModelDeux
        print("Appel de model 3 terminé")
        res_model2 = # TODO # fait passer self pour une instance de ModelDeux, le mro appelé 
        return res_model3+res_model2

In [ ]:
clf = ModelQuatre(33,34)
print("--------")
clf.fit(2)
print("--------")
clf.predict(3)

## La morale :
- Ne jamais utiliser super() en dehors du constructeur sauf si vous savez vraiment pourquoi c'est nécessaire. Préferez ClassParent.method(self, params)
- Toujours appeler super() dans le constructeur, de façon à ce que le constructeur de chaque classe ne soit appelé qu'une fois
- si vous voulez plus d'infos, super article (en anglais) https://fuhm.net/super-harmful/

# Héritage dynamique

Probleme : je veux créer une classe modele5, qui peut hériter soit de modele 1 soit de modele 2, et rajouter une méthode fit. Comment faire ?
- probleme courant quand on crée nos experiences en fonction de fichier de configurations.
Solution : constuire une méthode qui construit la classe en fonction de la classe parente demandée

In [ ]:
def model_5_builder(parent_class):
    # TODO
    return Model5

In [ ]:
print("---------- Instance with model 1 parent ------")
clf = model_5_builder(ModelUn)(33)
print("--------")
clf.fit(2)
print("--------")
clf.predict(3)

print("-------- Instance with model 2 parent --------")
clf = model_5_builder(ModelDeux)(33,34)
print("--------")
clf.fit(2)
print("--------")
clf.predict(3)

# Paramètres dynamiques

Probleme : comment faire une méthode qui peut accepter un nombre variable de paramètre ?
- probleme recurrent lorsque l'on fait des experiences decrites par des fichiers
- bonnes pratique pour ne pas avoir a modifier un code si on sait que le nom ou quantité de parametres change tout le temps.
- souvent necessaire pour la methode init pour quelle soit compatible avec super()

In [ ]:
def my_method():
    # TODO

In [ ]:
my_method(deux=2, trois=3, quaranteDeux = 'toto')

- Comment faire pour stocker dynamiquement un nombre de parametre variable dans l'init d'une classe ?

In [ ]:
class DynClass():
    def __init__(self, **args):
        # TODO

In [ ]:
my_cls = DynClass(deux=2, trois=3, quaranteDeux = 'toto')
my_cls.trois

Les classes en python fonctionnent en fait avec un dictionnaire qui contient tous leur parametres 

In [ ]:
my_cls.__dict__

# Bonus : les methodes particulières

- print personalisé des classes, plus utile pour le debugage :

In [ ]:
my_cls = DynClass(deux=2, trois=3, quaranteDeux = 'toto')
print(my_cls)

In [ ]:
my_cls

Cela ne nous apporte pas beaucoup d'information ! __repr__ et __str__ peuvent nous permettrent d'afficher des informations pertinentes à la place

In [ ]:
class DynClass():
    def __init__(self, **args):
        for key, value in args.items():
            self.__dict__[key]=value
        self.static_param = 4
    
    def __repr__(self):
        res = ''
        for param, value in self.__dict__.items():
            res += "Param "+ param +" : "+str(value)+'\n'
        return res
    
    def __str__(self):
        return "What a nice "+self.__class__.__name__+ " with "+str(len(self.__dict__.keys()))+' parameters !'

In [ ]:
my_cls = DynClass(deux=2, trois=3, quaranteDeux = 'toto')
print(my_cls)

In [ ]:
my_cls